# Custom solvers
In this notebook we will take a quick look on how the user may wrap an external solver for use with ``pyneqsys``.

In [ ]:
import warnings
import numpy as np
import scipy
from matplotlib.colors import Normalize
import matplotlib.cm as cm
import matplotlib.pyplot as plt
%matplotlib inline

We will use a 2 dimensional problem for illustration:

In [ ]:
def f(x, p):
    return [
        (-2*p[0] - 4*p[1]*x[0]*(-x[0]**2 + x[1]) + 2*x[0])**2,
         p[1]**2*(-2*x[0]**2 + 2*x[1])**2
    ]
    #pp1 = p[0]+p[1]+1
    #dx = x[0] - x[1]
    #exprs = [
    #    (dx**3 + 3*(x[0]-3*pp1/5)**2)/(1+x[1])*pp1,
    #    (dx**3 + 5*(x[1]-3*pp1/7)**2)/(1+x[0])*pp1
    #]
    # Differentiation of expressions above:
    exprs = [
        ((x[0] - x[1])**3 + 3*(-3*p[0]/5 - 3*p[1]/5 + x[0] - 3/5)**2)*(p[0] + p[1] + 1)**2*(-36*p[0]/5 - 36*p[1]/5 + 12*x[0] + 6*(x[0] - x[1])**2 - 36/5)/(x[1] + 1)**2 + 6*(x[0] - x[1])**2*((x[0] - x[1])**3 + 5*(-3*p[0]/7 - 3*p[1]/7 + x[1] - 3/7)**2)*(p[0] + p[1] + 1)**2/(x[0] + 1)**2 - 2*((x[0] - x[1])**3 + 5*(-3*p[0]/7 - 3*p[1]/7 + x[1] - 3/7)**2)**2*(p[0] + p[1] + 1)**2/(x[0] + 1)**3,
        -6*(x[0] - x[1])**2*((x[0] - x[1])**3 + 3*(-3*p[0]/5 - 3*p[1]/5 + x[0] - 3/5)**2)*(p[0] + p[1] + 1)**2/(x[1] + 1)**2 - 2*((x[0] - x[1])**3 + 3*(-3*p[0]/5 - 3*p[1]/5 + x[0] - 3/5)**2)**2*(p[0] + p[1] + 1)**2/(x[1] + 1)**3 + ((x[0] - x[1])**3 + 5*(-3*p[0]/7 - 3*p[1]/7 + x[1] - 3/7)**2)*(p[0] + p[1] + 1)**2*(-60*p[0]/7 - 60*p[1]/7 + 20*x[1] - 6*(x[0] - x[1])**2 - 60/7)/(x[0] + 1)**2
    ]
    return exprs
    

In [ ]:
def visualize(f, params, *, bounds=[(-2,2), (-1,3)], three_dim=False, ax2D=None, ax3D=None, plot3d_kw=None, fig=None):
    x = np.linspace(*bounds[0], 10)
    y = np.linspace(*bounds[1], 10)
    xx, yy = np.meshgrid(x, y, sparse=True)
    z = f([xx,yy], params)
    rms = np.sum(np.asarray(z)**2, axis=0)**0.5
    if ax2D is None and ax3D is None:
        ax2D = True
    
    
    if ax2D:
        if ax2D is None:
            fig, ax2D = plt.subplots(1,1)
        #h = ax2D.contourf(x,y, rms, cmap=m)
        h=ax2D.pcolormesh(x,y,rms, shading='auto')
        #cb = fig.colorbar(h)
        #cb.set_label("RMS")
        m = cm.ScalarMappable(cmap=h.cmap, norm=h.norm)
        colors=h.cmap(h.norm(rms))
    else:
        norm=Normalize(vmin=rms.min(), vmax=rms.max())
        colors=cm.viridis(norm(rms))
        m = cm.ScalarMappable(cmap=cm.viridis, norm=norm)
    m.set_array(rms)
    
    if ax3D:
        if ax3D is True:
            fig, ax3D = plt.subplots(1,1, subplot_kw=dict(projection='3d'))    
        s = ax3D.plot_surface(xx, yy, rms, facecolors=colors, **(plot3d_kw or {}))
    
    
        
    cb = fig.colorbar(m)
    cb.set_label("RMS")

In [ ]:
fig = plt.figure(figsize=(14,6))
tough_parameters = [1., 100.]
visualize(f, tough_parameters, ax2D=fig.add_subplot(1,2,1), ax3D=fig.add_subplot(1,2,2,projection='3d'), fig=fig)

In [ ]:
fig = plt.figure(figsize=(14,6))
easy_parameters=[1.0, 0.2]
visualize(f, easy_parameters, ax2D=fig.add_subplot(1,2,1), ax3D=fig.add_subplot(1,2,2,projection='3d'), fig=fig)

In [ ]:
from pyneqsys.symbolic import SymbolicSys
help(SymbolicSys.from_callback)

In [ ]:
neqsys = SymbolicSys.from_callback(f, 2, 2)
help(neqsys.solve)

In [ ]:
neqsys.solve([.3, .7], tough_parameters)

SciPy's could find the root, let's see how KINSOL from SUNDIALS fares:

In [ ]:
neqsys.solve([.3, .7], tough_parameters, solver='kinsol', mxiter=400)

In [ ]:
neqsys.solve([.3, .7], easy_parameters, solver='kinsol', mxiter=400)

No problem. In SciPy v0.17 a new pure-python least squares optimizer was introduced, let's wrap it for use within ``pyneqsys``:

In [ ]:
class SciPyLsq:
    def __init__(self, neqsys):
        self.neqsys = neqsys

    def __call__(self, x0, **kwargs):
        new_kwargs = kwargs.copy()
        if self.neqsys.band is not None:
            raise ValueError("Not supported (see SciPy docs)")
        new_kwargs['args'] = (self.neqsys.internal_params,)
        return scipy.optimize.least_squares(self.neqsys.f_cb, x0, jac=self.neqsys.j_cb, **new_kwargs)
result = neqsys.solve([.3, .7], tough_parameters, attached_solver=SciPyLsq)
print(result)

We can see that the wrapping is quite straightforward. (the solver can then be used with e.g. the symbolic facilities of ``pyneqsys``).

## Looking at some demo-solvers distributed with ``pyneqsys``
In ``pyneqsys.solvers`` there are some demo solvers provided (they are not "producation grade" but rather serves as API examples.

In [ ]:
#import pyneqsys.solvers   # uncomment to look at the source code
#pyneqsys.solvers??

We will plot the convergence behaviour of the solvers:

In [ ]:
def plot_convergence(attached_solver, plot_attr, params=()):
    import numpy as np
    import matplotlib.pyplot as plt
    %matplotlib inline
    x_history = np.array(attached_solver.history_x)
    plt.figure(figsize=(15, 3))
    plt.subplot(1, 4, 1)
    plt.plot(x_history[:, 0], x_history[:, 1]); plt.xlabel('x0'), plt.ylabel('x1')
    plt.subplot(1, 4, 2)
    plt.plot(neqsys.rms(x_history, params)); plt.xlabel('iteration'), plt.ylabel('RMS(residuals)')
    plt.subplot(1, 4, 3)
    plt.semilogy(range(15, len(x_history)), neqsys.rms(x_history[15:], params)); plt.xlabel('iteration'), plt.ylabel('RMS(residuals)')
    plt.subplot(1, 4, 4)
    plt.plot(np.asarray(getattr(attached_solver, plot_attr)))
    plt.ylabel(plot_attr)
    plt.xlabel('iteration')
    plt.tight_layout()

Let's start with a line-searching gradient descent solver:

In [ ]:
from pyneqsys.solvers import LineSearchingGradientDescentSolver as LSGD
lsgd = LSGD()
print(neqsys.solve([.3, .7], easy_parameters, maxiter=2500, attached_solver=lsgd))
plot_convergence(lsgd, 'history_rms_f', easy_parameters)

We can compare this with a conjugate gradient solver:

In [ ]:
from pyneqsys.solvers import PolakRibiereConjugateGradientSolver as CG
cg = CG(4)
print(neqsys.solve([.3, .7], easy_parameters, attached_solver=cg))
plot_convergence(cg, 'history_sn', easy_parameters)

One can also build generalizations of the solvers quite easily, here is a damped gradient descent solver with damping chosen from the iteration history:

In [ ]:
from pyneqsys.solvers import AutoDampedGradientDescentSolver as ADGD
adgd = ADGD(1e-2, 3e-2, 4, .5)
print(neqsys.solve([.3, .7], easy_parameters, maxiter=300, attached_solver=adgd))
plot_convergence(adgd, 'history_damping', easy_parameters)

this notebook hopefully shows that the API of ``pyneqsys`` is quite approachable.